## Config

In [87]:
### Configuration
# Dependencies
import os
import pandas as pd
import numpy as np
import summerplaylist

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

Opening browser for Spotify login...


## Log In

In [ ]:
# Authenticate to Spotify
spotify, userID, user_token = authenticateToSpotify(CLIENTID, CLIENTSECRET, REDIRECT_URI)

## Functions

In [73]:
# # Function to scrape list of artist names from MusicFestivalWizard
# def fetchMusicFestivalWizard():
#     session = HTMLSession()
#     r = session.get('https://www.musicfestivalwizard.com/all-festivals/')
#     artists = r.html.find('#artist', first=True).find('option')
#     artistList = [x.text for x in artists]
#     result = pd.DataFrame({'artistName':artistList})
#     result = result.loc[result.artistName!="All Artists"]
#     return result

In [74]:
# # Create function to search spotify based on artist name
# def fetchArtistInfo(artists):
#     try:
#         # Search spotify for artists based on name
#         possibleArtists, = spotify.search(artists['artistName'], types=['artist'], limit=1)
#         # Select the first search result
#         artists['artistID'] = possibleArtists.items[0].id
#         artists['artistPopularity'] = possibleArtists.items[0].popularity
#         artists['artistFollowers'] = possibleArtists.items[0].followers.total
#         artists['artistGenre'] = ", ".join(list(possibleArtists.items[0].genres))
#         result = artists
#     except:
#         artists['artistID'] = np.nan
#         artists['artistPopularity'] = np.nan
#         artists['artistFollowers'] = np.nan
#         artists['artistGenre'] = np.nan
#         result = artists
#     finally:
#         # Wait 2 seconds so we do not exceed our API call allowance
#         time.sleep(2)
#         return result

In [75]:
# # Define function that will get the top tracks for each artist in the list
# def fetchTopSongs(row):
#     try:
#         # Query the spotify API
#         topSongs = spotify.artist_top_tracks(row['artistID'], "US")
#         # Iterate over the tracks to find a list of songURIs
#         # Limit this to only those songs produced in the last few years; Use only the year (first four digits)
#         # Limit this to only three songs per artist
#         result = [x.uri for x in topSongs if datetime.strptime(x.album.release_date[:4], '%Y')>MINDATE][:3]
#     except: 
#         result = [np.nan]
#     finally:
#         # Wait 2 seconds so we do not exceed our API call allowance
#         time.sleep(2)
#         return result

In [129]:
# def getPlaylistID(playlistName):
#     with spotify.token_as(user_token):
#         # Check if playlist already exists; if not create a new one
#         userPlaylists = pd.DataFrame([{'id':x.id, 'name':x.name} for x in spotify.playlists(userID, limit=40).items])
#         if (userPlaylists['name'].eq(playlistName)).any():
#             # Create a playlist for the user
#             newPlaylist = spotify.playlist_create(userID, playlistName, public=False, description='Top three songs from new artists')
#             playlistID = newPlaylist.id
#         # Else just get playlistID
#         else:
#             playlistID = userPlaylists.loc[userPlaylists['name']==playlistName, 'id'][0]
#     return playlistID

In [112]:
# def addToPlaylist(df, playlistID):
#     with spotify.token_as(user_token):
#         # Counter
#         n = 0
#         # Add songs in chunks of 100
#         while n < len(df):
#             songsToAdd = df.iloc[n:(n+100)]
#             spotify.playlist_add(playlistID, list(songsToAdd['songURI']), position=None)
#             # Wait two seconds to avoid exceeding API limits
#             time.sleep(2)
#             # Increment counter
#             n += 100

## Collect new artists

In [119]:
# Collect names of new artists
artistsNew = fetchMusicFestivalWizard()

## Process new artists

In [125]:
# Filter out known artists who we have already listened to
if os.path.exists("artistsKnown.csv"):
    artistsKnown = pd.read_csv('artistsKnown.csv', index_col=0)
else:
    artistsKnown = pd.DataFrame(columns=['artistName', 'artistID', 'artistPopularity', 'artistFollowers', 'artistGenre'])
artistsNew = artistsNew.loc[~artistsNew.artistName.isin(artistsKnown['artistName'])]
# Search Spotify for new artist info
artistsNew = artistsNew.apply(fetchArtistInfo, axis=1)
# Ignore artists who did not turn up in Spotify
artistsNew = artistsNew.dropna(subset='artistID')
# Drop artists whose name was not an exact match with a known artist but who have the same artistID
artistsNew = artistsNew.loc[~artistsNew.artistID.isin(artistsKnown['artistID'])]

## Get recent songs

In [83]:
# Get their top three songs in last 5 years
songs = artistsNew.copy()
# Find a list of songURIs for each artist
songs['songURI'] = artistsNew.apply(fetchTopSongs, axis=1)
# Explode lists of songURIs to multiple rows
songs = songs.explode('songURI', ignore_index=True)
# Drop where songs could not be found
songs = songs.dropna()
# How many songs are in this round?
print(f"There are {len(songs)} about to be added")

There are 6 about to be added


## Put songs in playlist

In [114]:
# Warning: This will make a new playlist
playlistID = getPlaylistID('New Artists to Explore')
# Add songs to the playlist
addToPlaylist(songs, playlistID)

## Update the list of known artists

In [126]:
# Append artistsNew to artistsKnown.csv 
allArtists = pd.concat([artistsKnown, artistsNew], axis=0, ignore_index=True)
allArtists.to_csv('artistsKnown.csv')

## Create playlist to drop good songs into

In [118]:
# Cull exploreArtists into ArtistHasPotential playlist
winnersPlaylist = getPlaylistID('New Artists I Liked')
print(f'Now drop songs you like into this playlist: {winnersPlaylist}')

Now drop songs you like into this playlist: 5bZPe8Gkc9jUADiDXXlm4V
